In [1]:
import re
import pandas as pd
import numpy as np
import openpyxl as xl

### Структура полей таблицы

In [2]:
# открыли файл
journal_name = 'Журнал контрактов 28.07.2020.xlsm'
workbook = xl.load_workbook(journal_name)

# открыли лист
sheet_name = 'действующие 2020'
sheet = workbook[sheet_name]

In [3]:
fields_indices = {}

for (column_num, field_name) in enumerate(map(lambda x: x.value, sheet[1])):
    if type(field_name) == str and not field_name[:1:].isdigit():
        fields_indices[field_name] = column_num

tmp = {}
for (column_num, field_name) in enumerate(map(lambda x: x.value, sheet[2])):
    if type(field_name) == str:
        tmp.setdefault(field_name, []).append(column_num)
        fields_indices[field_name + ' ' + str(len(tmp[field_name]))] = column_num

In [4]:
fields_indices

{'№п/п': 0,
 'Наименование организации заказчика': 1,
 'Адрес, контакт': 2,
 '№ договора / контракта': 3,
 'Срок': 4,
 'Статус': 5,
 'ИНН': 6,
 'Внеш': 7,
 'ДТУ': 8,
 'ДТЛ': 9,
 '№ прот 1': 10,
 'Отб+Зак 1': 11,
 'С/А 1': 12,
 'сумма 1': 13,
 '№ прот 2': 15,
 'Отб+Зак 2': 16,
 'С/А 2': 17,
 'сумма 2': 19,
 '№ прот 3': 21,
 'Отб+Зак 3': 22,
 'С/А 3': 23,
 'сумма 3': 25,
 '№ прот 4': 27,
 'Отб+Зак 4': 28,
 'С/А 4': 29,
 'сумма 4': 30}

### Считываем таблицу

In [5]:
fields_types = {
    '№п/п': 'Int64',
    'Наименование организации заказчика': 'string',
    'Адрес, контакт': 'string',
    '№ договора / контракта': 'string',
    'Срок': 'string',
    'Статус': 'string',
    'ИНН': 'Int64',
    'Внеш': 'Int64',
    'ДТУ': 'Int64',
    'ДТЛ': 'Int64',
    '№ прот 1': 'string',
    'Отб+Зак 1': 'string',
    'С/А 1': 'string',
    'сумма 1': 'float64',
    '№ прот 2': 'string',
    'Отб+Зак 2': 'string',
    'С/А 2': 'string',
    'сумма 2': 'float64',
    '№ прот 3': 'string',
    'Отб+Зак 3': 'string',
    'С/А 3': 'string',
    'сумма 3': 'float64',
    '№ прот 4': 'string',
    'Отб+Зак 4': 'string',
    'С/А 4': 'string',
    'сумма 4': 'float64'
}

In [6]:
xldf = pd.read_excel(journal_name, sheet_name=sheet_name,
                     dtype=fields_types,
                     names=fields_indices.keys(), index_col=None,
                     usecols=fields_indices.values(),
                     skiprows=1, nrows=153)

In [7]:
xldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 26 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   №п/п                                151 non-null    Int64  
 1   Наименование организации заказчика  153 non-null    string 
 2   Адрес, контакт                      152 non-null    string 
 3   № договора / контракта              148 non-null    string 
 4   Срок                                133 non-null    string 
 5   Статус                              135 non-null    string 
 6   ИНН                                 150 non-null    Int64  
 7   Внеш                                0 non-null      Int64  
 8   ДТУ                                 85 non-null     Int64  
 9   ДТЛ                                 61 non-null     Int64  
 10  № прот 1                            111 non-null    string 
 11  Отб+Зак 1                           113 non-n

### Информация ИДК клиентов

In [8]:
def parse_contact_info(contact_info):
    address, postcode, contact = None, None, None
    try:
        address, contact = tuple(contact_info.split(';')[:2:])
    except Exception:
        address = contact_info
        contact = pd.NA
    
    try:
        postcode = int(re.search(r'\b\d{6}\b', address).group(0))
    except Exception:
        pass
    
    return address, postcode, contact

In [9]:
idc_info = pd.DataFrame(columns=[
    'address', 'contact', 'postcode',
    'DTU_num', 'DTL_num'
])

idc_info['DTU_num'] = xldf['ДТУ']
idc_info['DTL_num'] = xldf['ДТЛ']

contact_info = xldf['Адрес, контакт'].apply(parse_contact_info)

idc_info['address'] =\
    contact_info.apply(lambda x: x[0]).astype('string')
idc_info['postcode'] =\
    contact_info.apply(lambda x: x[1]).astype('Int32')
idc_info['contact'] =\
    contact_info.apply(lambda x: x[2]).astype('string')

idc_info.head()

,address,contact,postcode,DTU_num,DTL_num
0,"357101, Ставропольский край, город Невинномысс...",8-928-818-28-30 Елена Валер...,357101,4,<NA>
1,"357112, Ставропольский край, город Невинномысс...",8-928-820-68-17 Гл. медсестра. Ирина Анатоль...,357112,3,<NA>
2,"357100, Ставропольский край, город Невинномысс...",Татьяна 8-918-876-48-05,357100,1,<NA>
3,"357113, Ставропольский край, город Невинномысс...",Ольга Гавриловна Рентген лаборант 8-919-754-...,357113,5,<NA>
4,"357107 Ставропольский край, г.Невинномысск, ул...",Галина Викторовна 8-905-496-20-13,357107,16,<NA>


### Клиентская база

In [10]:
client = pd.DataFrame(columns=[
    'name', 'taxpayer_id'
])

client['name'] = xldf['Наименование организации заказчика']
client['taxpayer_id'] = xldf['ИНН']
    
client.head()

,name,taxpayer_id
0,ГБУЗ СК ГБ (ГП №2),2635221863
1,ГБУЗ СК ГБ (ГБ№2),<NA>
2,Дантист,<NA>
3,ГБУЗ СК ГБ (ГДБ),2631017874
4,"Невинномысский филиал АНМО ""СКККДЦ"", г.невинно...",2634049360


### Квартальная отчетность

In [11]:
quarter = {
    1: xldf[['№ прот 1', 'Отб+Зак 1', 'С/А 1', 'сумма 1']].copy(),
    2: xldf[['№ прот 2', 'Отб+Зак 2', 'С/А 2', 'сумма 2']].copy(),
    3: xldf[['№ прот 3', 'Отб+Зак 3', 'С/А 3', 'сумма 3']].copy(),
    4: xldf[['№ прот 4', 'Отб+Зак 4', 'С/А 4', 'сумма 4']].copy(),
}

tmp = [
    'protocol_num',
    'docs_status',
    'bill_num',
    'tender'
]

quarter[1].columns = tmp
quarter[2].columns = tmp
quarter[3].columns = tmp
quarter[4].columns = tmp

quarter[1] = quarter[1].dropna(how='all')
quarter[2] = quarter[2].dropna(how='all')
quarter[3] = quarter[3].dropna(how='all')
quarter[4] = quarter[4].dropna(how='all')

quarter[1]['year'] = 2019
quarter[2]['year'] = 2019
quarter[3]['year'] = 2019
quarter[4]['year'] = 2019

quarter[1]['quarter'] = 1
quarter[2]['quarter'] = 2
quarter[3]['quarter'] = 3
quarter[4]['quarter'] = 4

quarter[1]['client'] = quarter[1].index
quarter[2]['client'] = quarter[2].index
quarter[3]['client'] = quarter[3].index
quarter[4]['client'] = quarter[4].index

In [12]:
total_quarter_info = pd.DataFrame()
total_quarter_info = total_quarter_info.append(quarter[1])
total_quarter_info = total_quarter_info.append(quarter[2])
total_quarter_info = total_quarter_info.append(quarter[3])
total_quarter_info = total_quarter_info.append(quarter[4])

In [13]:
total_quarter_info = total_quarter_info[[
    'year',
    'quarter',
    'client',
    'protocol_num',
    'docs_status',
    'bill_num',
    'tender'
]]

In [14]:
total_quarter_info

,year,quarter,client,protocol_num,docs_status,bill_num,tender
0,2019,1,0,723,О,331,NaN
1,2019,1,1,728,О,331,NaN
2,2019,1,2,761,О,422,1200.0
3,2019,1,3,706,О,331,NaN
4,2019,1,4,708,О,322,10400.0
...,...,...,...,...,...,...,...
124,2019,4,124,<NA>,З,<NA>,NaN
131,2019,4,131,<NA>,З,<NA>,NaN
142,2019,4,142,<NA>,З,<NA>,NaN
143,2019,4,143,<NA>,З,<NA>,NaN


### Список договоров

In [15]:
doc = pd.DataFrame(columns=[
    'client_id',
    'type',
    'num',
    'begin',
    'end'
])

In [16]:
doc_titles = xldf['№ договора / контракта']
doc_ends = xldf['Срок']
doc_id = 0
for (client_id, titles) in filter(lambda x: pd.notnull(x[1]),
                                     enumerate(doc_titles)):
    begin = pd.NA
    end = pd.NA
    types = list(map(lambda x: x.lower().title(),
                     re.findall(r'договор|контракт', titles, flags=re.I)))
    
    titles_lst = list(map(lambda x: re.sub(r'\s+|\n+', ' ', x.strip()),
                      filter(lambda x: len(x) > 0,
                             re.split(r'договор|контракт', titles, flags=re.I))))
    
    for (doc_type, doc_title) in zip(types, titles_lst):
        num = list(map(lambda x: re.sub(r'№|\bот\b', '', x).strip(),
                       re.findall(r'^.+от|^.+$', doc_title)))[0]
        
        dates = re.findall(r'\d{1,2}[\.|/|\-]\d\d[\.|/|\-]\d{4}', doc_title)
        
        if len(dates) > 0:
            begin = dates[0]
        end = doc_ends[client_id]
        
        if pd.notnull(end):
            end = re.sub(r'00:00:00', '', end)    
        elif len(dates) == 2:
            end = dates[1]
        doc.loc[doc_id] = [client_id, doc_type, num, begin, end]
        doc_id = doc_id + 1

In [17]:
doc.head()

,client_id,type,num,begin,end
0,0,Контракт,286-О,30.03.2020,2021-03-31
1,1,Контракт,286-О,30.03.2020,2021-03-31
2,2,Договор,09/01-20 ИД,09.01.2020,2022-01-09
3,3,Контракт,286-О,30.03.2020,2021-03-31
4,4,Договор,72/01-20 ИД,09.01.2020,2020-12-31


In [18]:
for doc_id, date in filter(lambda x: pd.notnull(x[1]), enumerate(doc.end)):
    doc.end[doc_id] = re.sub(r'г', '', date)

for doc_id, date in filter(lambda x: pd.notnull(x[1]), enumerate(doc.end)):
    doc.end[doc_id] = '-'.join(date.strip().split('.')[::-1])

for doc_id, date in filter(lambda x: pd.notnull(x[1]), enumerate(doc.begin)):
    doc.begin[doc_id] = '-'.join(date.strip().split('.')[::-1])

In [19]:
doc.begin = doc.begin.astype('datetime64')
doc.end = doc.end.astype('datetime64')
doc.num = doc.num.astype('string')
doc.type = doc.type.astype('string')
doc.client_id = doc.client_id.astype('Int32')

In [20]:
doc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167 entries, 0 to 166
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   client_id  167 non-null    Int32         
 1   type       167 non-null    string        
 2   num        167 non-null    string        
 3   begin      164 non-null    datetime64[ns]
 4   end        154 non-null    datetime64[ns]
dtypes: Int32(1), datetime64[ns](2), string(2)
memory usage: 12.3 KB


### Сохраним все в .csv

In [21]:
client.to_csv('client.csv', header=False)

In [22]:
idc_info.to_csv('idc_client_info.csv', header=False)

In [23]:
doc.to_csv('doc.csv', header=False)

In [31]:
total_quarter_info.to_csv('total_quarter_info.csv', header=False, index=False)

In [30]:
total_quarter_info[total_quarter_info['quarter'] == 2].head(20)

,year,quarter,client,protocol_num,docs_status,bill_num,tender
0,2019,2,0,00114,О,<NA>,NaN
1,2019,2,1,00075,О,<NA>,NaN
2,2019,2,2,00146,О,734,1200.0
3,2019,2,3,00018,О,<NA>,NaN
4,2019,2,4,00068,О,602,10400.0
5,2019,2,5,00117,О,662,1200.0
6,2019,2,6,00109,О,650,1200.0
7,2019,2,7,00149,О,741,3600.0
8,2019,2,8,00118,О,663,2400.0
10,2019,2,10,00074,О,611,2400.0
